<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/mobilnet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Существует три типа методов тонкой настройки tenorflow:

*   Использование ТФ-стройный Строительство веса и хорошую сетевую структуру, ручная регулировка
* Использование ТФ-тонкий при условии train_image_classifier.py сценарий автоматической сборки, специфические методы здесь
* Используя tf.keras , процесс такой же, как и керас

Здесь в основном представим первый метод выше, обратите внимание:

* Tensorflow / models удален из мэйнфрейма tf после версии 1.0. Его необходимо загрузить вручную. Здесь находится тензор потока / models . Вы можете использовать git clone, чтобы загрузить его в локальный каталог. Используйте следующую команду, чтобы временно добавить его в путь поиска python .

In [0]:
import sys
sys.path.append("./models/research/slim")

* Файл контрольных точек сети предварительного обучения Tf-slim в файле tensorflow/models/research/slim, [общие веса предварительного обучения сети](http://link.zhihu.com/?target=https%3A//github.com/tensorflow/models/blob/master/research/slim/README.md)
* Файл контрольных точек сети предварительной подготовки мобильной сети указан более конкретно в файлах slim/nets/mobilenet и [Mobilenet](http://link.zhihu.com/?target=https%3A//github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/README.md).


# 2.1 Метод построения модели

Существует три метода для тензорного потока для восстановления структуры и весов модели из файла контрольных точек.После восстановления перечисленных здесь моделей вычисления прямого деривации могут быть непосредственно выполнены для прогнозного анализа

**1) Загрузите структуру графика напрямую, затем загрузите вес**

In [0]:
# import_meta_graph можно загрузить структуру графика непосредственно из метафайла
saver = tf.train.import_meta_graph(os.path.join(model_path,r"resnet_v2/model.ckpt-258931.meta"))

# allow_soft_placement автоматический выбор устройства
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    # latest_checkpoint Проверить файл контрольной точки, чтобы найти последнюю модель
    # restore Восстановить вес графика
    saver.restore(sess,tf.train.latest_checkpoint(r"./model/resnet_v2"))
    graph = sess.graph
    # Get_tensor_by_name Получить тензор по имени тензорного
    print(sess.run(graph.get_tensor_by_name("resnet_model/conv2d/kernel:0")))

**2) Сначала построить структуру графика, а затем загрузить вес**

In [0]:
# Временно добавить slim в путь поиска Python
import sys
sys.path.append("./models/research/slim")

# Импорт mobilenet_v2
from nets.mobilenet import mobilenet_v2
# Сбросить диаграмму 
tf.reset_default_graph()

# Импортировать mobilenet, сначала построить структуру графа.
‘’‘После загрузки tf.get_default_graph () содержит структуру графа расчета mobilenet. 
Вы можете использовать tf.get_collection (tf.GraphKeys.TRAINABLE_VARIABLES), 
чтобы сравнить разницу до и после reset_graph’‘’

images = tf.placeholder(tf.float32,(None,224,224,3))
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
    logits, endpoints = mobilenet_v2.mobilenet(images,depth_multiplier=1.4)

# Определить класс 'saver' для восстановления весов графа
saver = tf.train.Saver()
with tf.Session() as sess:
   # latest_checkpoint Проверить файл контрольной точки, чтобы найти последнюю модель
   # restore Восстановить вес графика
    saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))
   # get_tensor_by_name Получить тензор по имени тензорного
    print(sess.run(tf.get_default_graph().get_tensor_by_name("MobilenetV2/Conv/weights:0")).shape)


Выведите имя узла в структуре графа. Имя тензора должно иметь номер, например 0, для обозначения первого вывода вычислительного узла:

In [0]:
for var in tf.trainable_variables():
  print(var.name)


**3) замороженный вывод**

Файл pb помещает всю структуру значения переменной и графика вычислений в файл и преобразует переменную и значение в константу с помощью convert_variable_to_constants. При проверке модели входные данные нужно только перенаправить на выходной слой.

In [0]:
# Прочитайте сохраненный файл pb и проанализируйте его в соответствующем GraphDef Protocol Buffer.
gd =  tf.GraphDef.FromString(open('./model_ckpt/mobilenet_v2/mobilenet_v2_1.4_224_frozen.pb',"rb").read())
# import_graph_def Загрузить график, сохраненный в graphdef, в текущий граф, return_elements возвращает указанный тензор
inp, predictions = \
tf.import_graph_def(gd,return_elements=["input:0","MobilenetV2/Predictions/Reshape_1:0"])

# График расчета в это время может быть использован непосредственно для прогнозирования
# Вытащите картинку! wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg
from PIL import Image
img = np.array(Image.open('panda.jpg').resize((224, 224))).astype(np.float) / 128 - 1
# inp - это входные данные, для которых требуется канал, а predictions - это структура прогнозирования, которую необходимо выводить.
with tf.Session(graph=inp.graph) as sess:
    x = sess.run(predictions,feed_dict={inp:img.reshape(1,224,224,3)})

**2.2 Finetune Process**

1. Построить структуру графика, перехватить целевой тензор, добавить новый слой
2. Загрузить целевой тензорный вес
3. Тренировка нового слоя
4. Глобальная подстройка

**1) Построить структуру графика, перехватить целевой тензор, добавить новый слой**


Структура графика на этом этапе представляет собой структуру графика расчета мобильной сети, полученную методом **« сначала построение структуры графика, а затем загрузка веса » .**

In [0]:
tf.reset_default_graph()
# Построить расчетную диаграмму
images = tf.placeholder(tf.float32,(None,224,224,3))
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
    logits, endpoints = mobilenet_v2.mobilenet(images,depth_multiplier=1.4)

# Получить целевой тензор, добавить новый слой
with tf.variable_scope("finetune_layers"):
    # Получить целевой тензор и вынуть тензор указанного слоя в mobilenet
    mobilenet_tensor = tf.get_default_graph().get_tensor_by_name("MobilenetV2/expanded_conv_14/output:0")
    # Передаем тензор на новый слой
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_1")(mobilenet_tensor)
    # Наблюдаем, обновляется ли вес нового слоя tf.summary.histogram("conv2d_1",x)
    x = tf.nn.relu(x,name="relu_1")
    x = tf.layers.Conv2D(filters=256,kernel_size=3,name="conv2d_2")(x)
    x = tf.layers.Conv2D(10,3,name="conv2d_3")(x)
    predictions = tf.reshape(x, (-1,10))

Рассчитаем структуру графика:

![Рассчитаем структуру графика:](https://pic4.zhimg.com/80/v2-f745afae04881302990bdffe3c5c0bf7_hd.jpg)

Внутри красного поля находится структура сети Mobilenet. Второй фиолетовый узел сверху вниз - это узел «MobilenetV2/extended_conv_14/output». Видно, что он напрямую связан с finetune_layers.

**2) Загрузите целевые веса и обучите новые слои**

In [0]:
# one-hot кодирование
def to_categorical(data, nums):
    return np.eye(nums)[data]
# Произвольно генерировать данные
x_train = np.random.random(size=(141,224,224,3))
y_train = to_categorical(label_fake,10)

# Конфигурация условий обучения
## label Placeholder
y_label = tf.placeholder(tf.int32, (None,10))
## Собирайте переменные в пределах области видимости finetune_layers, обновляйте только вес добавленного слоя
train_var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="finetune_layers")
## Определение потери
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_label,logits=predictions)
## Определите метод оптимизации, используйте var_list, чтобы указать вес для обновления, и обновляйте только вес train_var в это время.
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss,var_list=train_var)
## Наблюдаем, обновляется ли вес нового слоя
tf.summary.histogram("mobilenet_conv8",tf.get_default_graph().get_tensor_by_name('MobilenetV2/expanded_conv_8/depthwise/depthwise_weights:0'))
tf.summary.histogram("mobilenet_conv9",tf.get_default_graph().get_tensor_by_name('MobilenetV2/expanded_conv_9/depthwise/depthwise_weights:0'))

## Объединить все резюме
merge_all = tf.summary.merge_all()

## Установите количество итераций и пакетов университетов
epochs = 10
batch_size = 16

# Получить функцию из указанного списка переменных var_list
def get_var_list(target_tensor=None):
    '''Получить функцию из указанного списка переменных var_list'''
    if target_tensor==None:
        target_tensor = r"MobilenetV2/expanded_conv_14/output:0"
    target = target_tensor.split("/")[1]
    all_list = []
    all_var = []
    # пройти все переменные, node.name получает имя переменной 
    # Не используйте tf.trainable_variables(), потому что moving_mean/variance, batchnorm не принадлежит к обучаемой переменной
    for var in tf.global_variables():
        if var != []:
            all_list.append(var.name)
            all_var.append(var)
    try:
        all_list = list(map(lambda x:x.split("/")[1],all_list))
        # Найти индекс соответствующей переменной области видимости
        ind = all_list[::-1].index(target)
        ind = len(all_list) -  ind - 1
        print(ind)
        del all_list
        return all_var[:ind+1]
    except:
        print("target_tensor is not exist!")

Имя целевого тензора, чтобы получить список переменных, которые необходимо загрузить весовые коэффициенты из файла var_list
target_tensor = "MobilenetV2/expanded_conv_14/output:0"
var_list = get_var_list(target_tensor)
saver = tf.train.Saver(var_list=var_list)

# Загрузите веса в файл и обучите новый слой
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    writer = tf.summary.FileWriter(r"./logs", sess.graph)
## Параметры инициализации: загрузка весов из файла train_var. Использовать функцию инициализации.
    sess.run(tf.variables_initializer(var_list=train_var))
    saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))
    
    for i in range(2000):
        start = (i*batch_size) % x_train.shape[0]
        end = min(start+batch_size, x_train.shape[0])
        _, merge, losses = sess.run([train_step,merge_all,loss],\
                             feed_dict={images:x_train[start:end],\
                                        y_label:y_train[start:end]})
        if i%100==0:
           writer.add_summary(merge, i)

**Особенности инициализации веса:**

1. Сначала используйте глобальную инициализацию *tf.global_variables_initializer()*, а затем используйте порядок saver.restore, который не может быть неправильным, иначе загруженный вес будет переинициализирован.

In [0]:
sess.run(tf.global_variables_initializer())
saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))

2. Сначала используйте saver.restore для загрузки весов из модели, а затем используйте *tf.variable_initializaer()* для инициализации указанного var_list, порядок можно изменить.

In [0]:
saver.restore(sess,tf.train.latest_checkpoint("./model_ckpt/mobilenet_v2"))
sess.run(tf.variables_initializer(var_list=train_var))

3. Первые два метода также инициализируют переменные для бесполезных узлов и должны заранее выполнить операцию saver.restore, что означает, что для обеспечения того, чтобы процесс finetune не сообщал об ошибке, требуется две операции save.restore. Теперь вы можете составить var_list, отфильтровав все переменные, которые должны загрузить веса из файла, а затем определить saver = tf.train.Saver (var_list) для выборочной загрузки переменных.

**В приведенном выше коде используется третий метод, разница между тремя вышеуказанными методами инициализации может быть тщательно понята. Посмотрите на скриншот ниже.**

![alt text](https://pic4.zhimg.com/80/v2-812216ca5067586fd223b5cee6f3571b_hd.jpg)